<a href="https://colab.research.google.com/github/HiroshiATakano/QC_Kentei/blob/main/Level1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import polars as pl

# 与えられたデータ
data = {
    'Factor_A': [1, 1, 1, 1, 2, 2, 2, 2],  # 因子A
    'Factor_B': [1, 1, 2, 2, 1, 1, 2, 2],  # 因子B
    'AxB': [1, 1, 2, 2, 2, 2, 1, 1],       # A×B
    'AxC': [1, 2, 1, 2, 2, 1, 2, 1],       # A×C
    'Factor_C': [1, 2, 1, 2, 1, 2, 1, 2],  # 因子C
    'Residual': [1,2,2,1,1,2,2,1],                        # 誤差
    'Factor_D': [1,2,2,1,2,1,1,2],
    'Data': [50, 16, 24, 46, 66, 88, 52, 34]  # データ
}

# DataFrameを作成
df = pd.DataFrame(data)
# PolarsでDataFrameを作成
df2 = pl.DataFrame(data)

# データを表示
print(df2)


shape: (8, 8)
┌──────────┬──────────┬─────┬─────┬──────────┬──────────┬──────────┬──────┐
│ Factor_A ┆ Factor_B ┆ AxB ┆ AxC ┆ Factor_C ┆ Residual ┆ Factor_D ┆ Data │
│ ---      ┆ ---      ┆ --- ┆ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---  │
│ i64      ┆ i64      ┆ i64 ┆ i64 ┆ i64      ┆ i64      ┆ i64      ┆ i64  │
╞══════════╪══════════╪═════╪═════╪══════════╪══════════╪══════════╪══════╡
│ 1        ┆ 1        ┆ 1   ┆ 1   ┆ 1        ┆ 1        ┆ 1        ┆ 50   │
│ 1        ┆ 1        ┆ 1   ┆ 2   ┆ 2        ┆ 2        ┆ 2        ┆ 16   │
│ 1        ┆ 2        ┆ 2   ┆ 1   ┆ 1        ┆ 2        ┆ 2        ┆ 24   │
│ 1        ┆ 2        ┆ 2   ┆ 2   ┆ 2        ┆ 1        ┆ 1        ┆ 46   │
│ 2        ┆ 1        ┆ 2   ┆ 2   ┆ 1        ┆ 1        ┆ 2        ┆ 66   │
│ 2        ┆ 1        ┆ 2   ┆ 1   ┆ 2        ┆ 2        ┆ 1        ┆ 88   │
│ 2        ┆ 2        ┆ 1   ┆ 2   ┆ 1        ┆ 2        ┆ 1        ┆ 52   │
│ 2        ┆ 2        ┆ 1   ┆ 1   ┆ 2        ┆ 1        ┆ 2        ┆ 34   

In [54]:
def calc_ss(df: pl.DataFrame, factor: str, data_col: str, total_mean: float) -> float:
    """
    指定した因子(factor)における平方和を計算する関数。
    例：Factor_A, Factor_B, ...
    """
    # 1グループの統計量
    df_grp1 = df.filter(pl.col(factor) == 1)
    mean_1 = df_grp1.select(pl.col(data_col).mean()).item()
    n1 = df_grp1.height

    # 2グループの統計量
    df_grp2 = df.filter(pl.col(factor) == 2)
    mean_2 = df_grp2.select(pl.col(data_col).mean()).item()
    n2 = df_grp2.height

    # 平方和の計算
    ss = n1 * ((mean_1 - total_mean) ** 2) + n2 * ((mean_2 - total_mean) ** 2)
    return ss

# 総平方和 (Total Sum of Squares)
total_mean = df['Data'].mean()
total_ss = sum((df['Data'] - total_mean) ** 2)

# 各因子の平方和を計算
ss_A = calc_ss(df2, "Factor_A", "Data", total_mean)
ss_B = calc_ss(df2, "Factor_B", "Data", total_mean)
ss_C = calc_ss(df2, "Factor_C", "Data", total_mean)
ss_D = calc_ss(df2, "Factor_D", "Data", total_mean)

# 交互作用の平方和
ss_AxB = calc_ss(df2, "AxB", "Data", total_mean)
ss_AxC = calc_ss(df2, "AxC", "Data", total_mean)

# 残差の平方和 (Residual Sum of Squares)
ss_residual = total_ss - (ss_A + ss_B + ss_C + ss_D + ss_AxB + ss_AxC)

print(f"総平方和: {total_ss}")

print("因子Aの平方和:", ss_A)
print("因子Bの平方和:", ss_B)
print("因子Cの平方和:", ss_C)
print("因子Dの平方和:", ss_D)

print("AxBの平方和:", ss_AxB)
print("AxCの平方和:", ss_AxC)

print("誤差の平方和 (Residual SS):", ss_residual)


総平方和: 3736.0
因子Aの平方和: 1352.0
因子Bの平方和: 512.0
因子Cの平方和: 8.0
因子Dの平方和: 1152.0
AxBの平方和: 648.0
AxCの平方和: 32.0
誤差の平方和 (Residual SS): 32.0


In [62]:
# 各項目の自由度 (df)
# ここでは各因子が2水準の場合とし、自由度は水準数-1
df_A   = 2 - 1
df_B   = 2 - 1
df_C   = 2 - 1
df_D   = 2 - 1
df_AxB = 2 - 1  # 交互作用も各群が2水準の場合
df_AxC = 2 - 1

# 残差の自由度は総データ数から、各効果の水準数の和（※単純加法の場合）
# ※ここでは単純な計算例として、総データ数 - (効果の数)
# 実際のモデルでは、各因子・交互作用の水準の重なり等により異なるので注意
n_total = df2.height
df_total = n_total - 1
df_model = df_A + df_B + df_C + df_D + df_AxB + df_AxC
df_residual = df_total - df_model

# 平均平方 (Mean Square: MS) の計算
ms_A   = ss_A   / df_A
ms_B   = ss_B   / df_B
ms_C   = ss_C   / df_C
ms_D   = ss_D   / df_D
ms_AxB = ss_AxB / df_AxB
ms_AxC = ss_AxC / df_AxC
ms_residual = ss_residual / df_residual

# F値の計算（各効果のMSを誤差のMSで割る）
F_A   = ms_A   / ms_residual
F_B   = ms_B   / ms_residual
F_C   = ms_C   / ms_residual
F_D   = ms_D   / ms_residual
F_AxB = ms_AxB / ms_residual
F_AxC = ms_AxC / ms_residual

# 分散分析表 (ANOVA table) の作成
anova_table = pl.DataFrame({
    "Source": ["Factor_A", "Factor_B", "Factor_C", "Factor_D", "AxB", "AxC", "Residual", "Total"],
    "SS":      [ss_A, ss_B, ss_C, ss_D, ss_AxB, ss_AxC, ss_residual, total_ss],
    "df":      [df_A, df_B, df_C, df_D, df_AxB, df_AxC, df_residual, df_total],
    "MS":      [ms_A, ms_B, ms_C, ms_D, ms_AxB, ms_AxC, ms_residual, None],
    "F":       [F_A, F_B, F_C, F_D, F_AxB, F_AxC, None, None]
})

print(anova_table)

shape: (8, 5)
┌──────────┬────────┬─────┬────────┬───────┐
│ Source   ┆ SS     ┆ df  ┆ MS     ┆ F     │
│ ---      ┆ ---    ┆ --- ┆ ---    ┆ ---   │
│ str      ┆ f64    ┆ i64 ┆ f64    ┆ f64   │
╞══════════╪════════╪═════╪════════╪═══════╡
│ Factor_A ┆ 1352.0 ┆ 1   ┆ 1352.0 ┆ 42.25 │
│ Factor_B ┆ 512.0  ┆ 1   ┆ 512.0  ┆ 16.0  │
│ Factor_C ┆ 8.0    ┆ 1   ┆ 8.0    ┆ 0.25  │
│ Factor_D ┆ 1152.0 ┆ 1   ┆ 1152.0 ┆ 36.0  │
│ AxB      ┆ 648.0  ┆ 1   ┆ 648.0  ┆ 20.25 │
│ AxC      ┆ 32.0   ┆ 1   ┆ 32.0   ┆ 1.0   │
│ Residual ┆ 32.0   ┆ 1   ┆ 32.0   ┆ null  │
│ Total    ┆ 3736.0 ┆ 7   ┆ null   ┆ null  │
└──────────┴────────┴─────┴────────┴───────┘


In [63]:
# --- プール処理 ---
# 因子CとAxCの効果を誤差にプールする
ss_residual_pool = ss_residual + ss_C + ss_AxC
df_residual_pool = df_residual + df_C + df_AxC

# --- 残差の平均平方 (MS) の計算 ---
ms_residual_pool = ss_residual_pool / df_residual_pool

# --- F値の計算 ---
F_A   = np.round(ms_A   / ms_residual_pool,2)
F_B   = np.round(ms_B   / ms_residual_pool,2)
F_D   = np.round(ms_D   / ms_residual_pool,2)
F_AxB = np.round(ms_AxB / ms_residual_pool,2)

# --- 分散分析表 (ANOVA table) の作成 ---
# プール後のANOVA表では、因子CとAxCは誤差に含まれるため、表示項目はA, B, D, AxB, Residual, Total
anova_table = pl.DataFrame({
    "Source": ["Factor_A", "Factor_B", "Factor_D", "AxB", "Residual (Pooled)", "Total"],
    "SS":      [ss_A, ss_B, ss_D, ss_AxB, ss_residual_pool, total_ss],
    "df":      [df_A, df_B, df_D, df_AxB, df_residual_pool, df_total],
    "MS":      [ms_A, ms_B, ms_D, ms_AxB, ms_residual_pool, None],
    "F":       [F_A, F_B, F_D, F_AxB, None, None]
})


print(anova_table)


shape: (6, 5)
┌───────────────────┬────────┬─────┬────────┬───────┐
│ Source            ┆ SS     ┆ df  ┆ MS     ┆ F     │
│ ---               ┆ ---    ┆ --- ┆ ---    ┆ ---   │
│ str               ┆ f64    ┆ i64 ┆ f64    ┆ f64   │
╞═══════════════════╪════════╪═════╪════════╪═══════╡
│ Factor_A          ┆ 1352.0 ┆ 1   ┆ 1352.0 ┆ 56.33 │
│ Factor_B          ┆ 512.0  ┆ 1   ┆ 512.0  ┆ 21.33 │
│ Factor_D          ┆ 1152.0 ┆ 1   ┆ 1152.0 ┆ 48.0  │
│ AxB               ┆ 648.0  ┆ 1   ┆ 648.0  ┆ 27.0  │
│ Residual (Pooled) ┆ 72.0   ┆ 3   ┆ 24.0   ┆ null  │
│ Total             ┆ 3736.0 ┆ 7   ┆ null   ┆ null  │
└───────────────────┴────────┴─────┴────────┴───────┘


In [57]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 因子A、因子B、因子C、および反復（R=2）の設定
levels_A = 3  # 因子Aの水準数
levels_B = 2  # 因子Bの水準数
levels_C = 3  # 因子Cの水準数
repeats = 2   # 反復回数

# 因子A、B、Cの水準の組み合わせを作成
A = np.tile(np.repeat(np.arange(levels_A), levels_B * levels_C * repeats), 1)
B = np.tile(np.repeat(np.arange(levels_B), levels_C * repeats), levels_A)
C = np.tile(np.arange(levels_C), levels_A * levels_B * repeats)
R = np.tile(np.arange(repeats), levels_A * levels_B * levels_C)

# サンプルデータ（ランダム生成された応答変数y）
np.random.seed(42)
y = np.random.randn(len(A)) * 5 + 10  # 平均10、標準偏差5のランダムデータ

# データフレームを作成
data = pd.DataFrame({
    'A': A,
    'B': B,
    'C': C,
    'R': R,
    'y': y
})

# 'A', 'B', 'C'をカテゴリカル変数に変換
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')

# 分散分析モデルの定義 (A, B, Cとその相互作用を含む)
model = ols('y ~ A + B + C + A:B + A:C + B:C', data=data).fit()

# 分散分析 (ANOVA) テーブル
anova_table = sm.stats.anova_lm(model, typ=2)

# 結果の表示
print("サンプルデータ:")
print(data)
print("\n分散分析 (ANOVA) テーブル:")
print(anova_table)

# 平方和の取得
ssq_A = anova_table['sum_sq']['A']
ssq_B = anova_table['sum_sq']['B']
ssq_C = anova_table['sum_sq']['C']

print(f"\n因子Aの平方和: {ssq_A}")
print(f"因子Bの平方和: {ssq_B}")
print(f"因子Cの平方和: {ssq_C}")


サンプルデータ:
    A  B  C  R          y
0   0  0  0  0  12.483571
1   0  0  1  1   9.308678
2   0  0  2  0  13.238443
3   0  0  0  1  17.615149
4   0  0  1  0   8.829233
5   0  0  2  1   8.829315
6   0  1  0  0  17.896064
7   0  1  1  1  13.837174
8   0  1  2  0   7.652628
9   0  1  0  1  12.712800
10  0  1  1  0   7.682912
11  0  1  2  1   7.671351
12  1  0  0  0  11.209811
13  1  0  1  1   0.433599
14  1  0  2  0   1.375411
15  1  0  0  1   7.188562
16  1  0  1  0   4.935844
17  1  0  2  1  11.571237
18  1  1  0  0   5.459880
19  1  1  1  1   2.938481
20  1  1  2  0  17.328244
21  1  1  0  1   8.871118
22  1  1  1  0  10.337641
23  1  1  2  1   2.876259
24  2  0  0  0   7.278086
25  2  0  1  1  10.554613
26  2  0  2  0   4.245032
27  2  0  0  1  11.878490
28  2  0  1  0   6.996807
29  2  0  2  1   8.541531
30  2  1  0  0   6.991467
31  2  1  1  1  19.261391
32  2  1  2  0   9.932514
33  2  1  0  1   4.711445
34  2  1  1  0  14.112725
35  2  1  2  1   3.895782

分散分析 (ANOVA) テーブル:
         

In [58]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# データの作成
data_dict = {
    'A': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3] * 2,
    'B': [1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2] * 2,
    'C': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3] * 2,
    'R': [1] * 18 + [2] * 18,
    'y': [
        7, 11, 11, 8, 20, 29, 13, 21, 21, 17, 26, 26, 21, 31, 25, 30, 33, 21,
        14, 20, 22, 10, 18, 22, 20, 20, 26, 24, 31, 33, 21, 26, 18, 28, 28, 22
    ]
}

# データフレームに変換
data = pd.DataFrame(data_dict)

# 因子A、B、Cをカテゴリカル変数に変換
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')

# 分散分析モデルの定義
model = ols('y ~ A + B + C + A:B + A:C + B:C', data=data).fit()

# 分散分析 (ANOVA) テーブル
anova_table = sm.stats.anova_lm(model, typ=2)

# 結果の表示
print("分散分析 (ANOVA) テーブル:")
print(anova_table)

# 平方和の取得
ssq_A = anova_table['sum_sq']['A']
ssq_B = anova_table['sum_sq']['B']
ssq_C = anova_table['sum_sq']['C']

print(f"\n因子Aの平方和: {ssq_A}")
print(f"因子Bの平方和: {ssq_B}")
print(f"因子Cの平方和: {ssq_C}")


分散分析 (ANOVA) テーブル:
              sum_sq    df          F    PR(>F)
A         572.666667   2.0  15.774624  0.000056
B         169.000000   1.0   9.310518  0.005854
C         256.500000   2.0   7.065526  0.004265
A:B        12.666667   2.0   0.348915  0.709287
A:C       273.333333   4.0   3.764608  0.017655
B:C         3.500000   2.0   0.096411  0.908473
Residual  399.333333  22.0        NaN       NaN

因子Aの平方和: 572.6666666666655
因子Bの平方和: 168.9999999999992
因子Cの平方和: 256.4999999999987


In [59]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# データの作成
data_dict = {
    'A': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3] * 2,
    'B': [1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2] * 2,
    'C': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3] * 2,
    'R': [1] * 18 + [2] * 18,
    'y': [
        7, 11, 11, 8, 20, 29, 13, 21, 21, 17, 26, 26, 21, 31, 25, 30, 33, 21,
        14, 20, 22, 10, 18, 22, 20, 20, 26, 24, 31, 33, 21, 26, 18, 28, 28, 22
    ]
}

# データフレームに変換
data = pd.DataFrame(data_dict)

# 因子A、B、Cをカテゴリカル変数に変換
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')

# 分散分析モデルの定義
model = ols('y ~ A + B + C + A:B + A:C + B:C', data=data).fit()

# 分散分析 (ANOVA) テーブル
anova_table = sm.stats.anova_lm(model, typ=2)

# 結果の表示
print("分散分析 (ANOVA) テーブル:")
print(anova_table)

# 平方和の取得
ssq_A = anova_table['sum_sq']['A']  # 一次誤差に関連する平方和
ssq_B = anova_table['sum_sq']['B']  # 二次誤差に関連する平方和
ssq_C = anova_table['sum_sq']['C']  # 三次誤差に関連する平方和

ssq_AB = anova_table['sum_sq']['A:B']  # AとBの交互作用
ssq_AC = anova_table['sum_sq']['A:C']  # AとCの交互作用
ssq_BC = anova_table['sum_sq']['B:C']  # BとCの交互作用

# 残差
ssq_residual = anova_table['sum_sq']['Residual']  # 残差

# 一次誤差の平方和（Aの影響）
ssq_once = ssq_A + ssq_AB + ssq_AC

# 二次誤差の平方和（B, Cの交互作用）
ssq_twice = ssq_B + ssq_BC

# 三次誤差の平方和（Cの影響 + 残差）
ssq_thrice = ssq_C + ssq_residual

# 結果の表示
print(f"\n一次誤差の平方和: {ssq_once}")
print(f"二次誤差の平方和: {ssq_twice}")
print(f"三次誤差の平方和: {ssq_thrice}")


分散分析 (ANOVA) テーブル:
              sum_sq    df          F    PR(>F)
A         572.666667   2.0  15.774624  0.000056
B         169.000000   1.0   9.310518  0.005854
C         256.500000   2.0   7.065526  0.004265
A:B        12.666667   2.0   0.348915  0.709287
A:C       273.333333   4.0   3.764608  0.017655
B:C         3.500000   2.0   0.096411  0.908473
Residual  399.333333  22.0        NaN       NaN

一次誤差の平方和: 858.6666666666652
二次誤差の平方和: 172.4999999999992
三次誤差の平方和: 655.8333333333321


In [60]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# データの作成
data_dict = {
    'A': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3] * 2,
    'B': [1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2] * 2,
    'C': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3] * 2,
    'R': [1] * 18 + [2] * 18,
    'y': [
        7.0, 11.0, 11.0, 8.0, 20.0, 29.0, 13.0, 21.0, 21.0, 17.0, 26.0, 26.0, 21.0, 31.0, 25.0, 30.0, 33.0, 21.0,
        14.0, 20.0, 22.0, 10.0, 18.0, 22.0, 20.0, 20.0, 26.0, 24.0, 31.0, 33.0, 21.0, 26.0, 18.0, 28.0, 28.0, 22.0
    ]
}

# データフレームに変換
data = pd.DataFrame(data_dict)

# カテゴリカル変数に変換
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')
data['R'] = data['R'].astype('category')

# 従属変数yを数値型に変換
data['y'] = pd.to_numeric(data['y'], errors='coerce')

# NaN の確認
nan_count = data.isnull().sum()
print("NaN のカウント:\n", nan_count)

# 無効な値が含まれているか確認
valid_data = data.select_dtypes(include=[np.number])  # 数値型の列を選択
if np.isfinite(valid_data.values).all():
    print("無効な値は含まれていません。")
else:
    print("無効な値が含まれています。")

# 分散分析モデルの定義
model = ols('y ~ A * B * C * R', data=data).fit()

# 分散分析 (ANOVA) テーブル
anova_table = sm.stats.anova_lm(model, typ=3)

# 結果の表示
print("\n分散分析 (ANOVA) テーブル:")
print(anova_table)

# 一次、二次、三次誤差の平方和を計算
ssq_AR = anova_table['sum_sq']['A:R']  # 一次誤差（A × R）
ssq_BR = anova_table['sum_sq']['B:R']  # 二次誤差（B × R）
ssq_CR = anova_table['sum_sq']['C:R']  # 三次誤差（C × R）

print(f"\n一次誤差（A × R）の平方和: {ssq_AR}")
print(f"二次誤差（B × R）の平方和: {ssq_BR}")
print(f"三次誤差（C × R）の平方和: {ssq_CR}")


NaN のカウント:
 A    0
B    0
C    0
R    0
y    0
dtype: int64
無効な値は含まれていません。


/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# データの作成
data_dict = {
    'A': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3] * 2,
    'B': [1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2] * 2,
    'C': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3] * 2,
    'R': [1] * 18 + [2] * 18,
    'y': [
        7, 11, 11, 8, 20, 29, 13, 21, 21, 17, 26, 26, 21, 31, 25, 30, 33, 21,
        14, 20, 22, 10, 18, 22, 20, 20, 26, 24, 31, 33, 21, 26, 18, 28, 28, 22
    ]
}

# データフレームに変換
data = pd.DataFrame(data_dict)

# カテゴリカル変数に変換
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')
data['R'] = data['R'].astype('category')

# 従属変数yを数値型に変換（必要に応じて）
data['y'] = pd.to_numeric(data['y'], errors='coerce')

# 正規性の確認
sns.histplot(data['y'], kde=True)
plt.title('yのヒストグラム')
plt.show()

# Shapiro-Wilk検定
shapiro_test = stats.shapiro(data['y'])
print(f"Shapiro-Wilk検定: 統計量={shapiro_test.statistic}, p値={shapiro_test.pvalue}")

# Levene検定（等分散性の確認）
levene_test = stats.levene(data['y'][data['A'] == 1],
                            data['y'][data['A'] == 2],
                            data['y'][data['A'] == 3])
print(f"Levene検定: 統計量={levene_test.statistic}, p値={levene_test.pvalue}")

# データの対数変換（必要な場合）
data['log_y'] = np.log(data['y'])

# 対数変換後の正規性の確認
sns.histplot(data['log_y'], kde=True)
plt.title('対数変換後のyのヒストグラム')
plt.show()

# 分散分析モデルの定義
model = ols('log_y ~ A * B * C * R', data=data).fit()

# 分散分析 (ANOVA) テーブルをタイプ3で計算
anova_table = sm.stats.anova_lm(model, typ=3)

# 結果の表示
print("\n分散分析 (ANOVA) テーブル:")
print(anova_table)


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# サンプルデータの作成
np.random.seed(0)  # 再現性のため乱数シードを固定
levels_A = ['A1', 'A2', 'A3']  # 因子Aの水準
levels_B = ['B1', 'B2']        # 因子Bの水準
levels_C = ['C1', 'C2', 'C3']  # 因子Cの水準
repeats = 5                    # 反復数を増加

# 各因子の水準を組み合わせたデータフレームを作成
data = pd.DataFrame([
    [a, b, c, np.random.normal(loc=10, scale=2)]
    for a in levels_A
    for b in levels_B
    for c in levels_C
    for _ in range(repeats)
], columns=['A', 'B', 'C', 'y'])

# モデルの定義と分散分析
model = ols('y ~ A * B * C', data=data).fit()  # Rを除外
anova_table = sm.stats.anova_lm(model, typ=3)

# 結果の表示
print(anova_table)


In [ ]:
data